# COGS 118B - Data Wrangling

# Names


- Ana Maria Baboescu
- Bradley Grace
- Fatima Enriquez
- Lucy Giang
- Stephanie Frianeza

In [ ]:
import pandas as pd
import numpy as np


# FMA - Bradley
Below code displays the updated dataset with the wanted variables

# Spotify - Lucy
Below code displays the updated dataset with the wanted variables

# 500 Greatest Songs - Fatima
Below code displays the updated dataset with the wanted variables


# Prediction of Music - Ana & Stephanie
Below code displays the updated dataset with the wanted variables


# Compiled Data - Everyone

Here we compile all the datasets together for Monday meeting


# EDA - TBD




# Program - TBD

# Footnotes
1. <a name="cite_note-1"></a> [^](#cite_ref-1) Chillar, Snigdha et al. (May 2019) Music Genre Classification using Machine Learning Algorithms: A comparison. *International Research Journal of Engineering and Technology (IRJET)*. https://d1wqtxts1xzle7.cloudfront.net/59934287/IRJET-V6I517420190704-120568-1u4iafr-libre.pdf?1562308085=&response-content-disposition=inline%3B+filename%3DIRJET_Music_Genre_Classification_using_M.pdf&Expires=1708407619&Signature=WRJ6JnCTvv8fyWmo~A-SzVQ2DRT77pSZFX8tmlz7YCCB7J5ZMkrJpugkiwlJT7DoaCR-d2jI6IfGjgYXI9-EsJlVANQ~gJY04gUz9H4zkZG-HiyimSeXcAkK58Rqp06qgvlu-yx5zcM1wNxrnhgASBQEvVBkhyQMxETbkgtLYzw40gKVZFeioo0Qjj7aqC-YDzGwlzlnXhNss4xlBsBj7PdyTFgGu2cM8ky8g3XsqA1yIIuiO4cr0SEcCJU8orRATVTpB388Nud0GvNiGOz6DAhpoCBLEqrsYxFHk2jOut7x1TPs91ECXgG1SwjPE03vlYoeAMEkdV5pasBm1yZDeQ__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA

2. <a name="cite_note-2"></a> [^](#cite_ref-2) Bahuleyan, Hareesh( 3 April 2018) Music Genre Classification using Machine Learning Techniques. Cornell University. https://doi.org/10.48550/arXiv.1804.01149

3. <a name="cite_note-3"></a> [^](#cite_ref-3) Wohlwend, Brandon (23 July 2023) Decision Tree, Random Forest, and XGBoost: An Exploration into the Heart of Machine Learning. Medium. https://medium.com/@brandon93.w/decision-tree-random-forest-and-xgboost-an-exploration-into-the-heart-of-machine-learning-90dc212f4948#:~:text=Random%20Forests%20build%20on%20this,that%20often%20outperforms%20many%20others.
